# Tensorflow GPU connection problem

Need to install tensorflow with pip. https://www.tensorflow.org/install/pip

Since installing with conda, we cannot detect GPU with below command.

https://www.tensorflow.org/guide/gpu 


In [2]:
import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()
import tensorflow as tf2
print ('tensorflow ver:', tf2.__version__)

# Return a list of physical devices visible to the host runtime.
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print("Num GPUs Available: ", len(tf2.config.list_physical_devices('GPU')))
tf.config.list_physical_devices('GPU')

tensorflow ver: 2.10.0
Num GPUs Available:  8
Num GPUs Available:  8


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:4', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:5', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:6', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:7', device_type='GPU')]

In [3]:
## Need to assign gpu number first (after checking memory usage with 'nvidia-smi' on terminal)
gpu_num = 0
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_num)

# Specifies which PhysicalDevice objects are visible to the runtime. TensorFlow will only allocate memory and place operations on visible physical devices, as otherwise no LogicalDevice will be created on them. By default all discovered devices are marked as visible.
gpus = tf.config.list_physical_devices('GPU')
# tf.config.set_visible_devices(gpus, 'GPU')
tf.config.set_visible_devices(gpus[0], 'GPU')

In [4]:
logical_gpus = tf.config.list_logical_devices('GPU')
print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")

8 Physical GPUs, 1 Logical GPU


2022-10-28 09:39:31.352330: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-28 09:39:41.959369: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30975 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:06:00.0, compute capability: 7.0


# Internal cuDNN launch failure problem

```
---------------------------------------------------------------------------
InternalError                             Traceback (most recent call last)
/nethome/jhan405/cbcv/tft_google_ver4/main.ipynb Cell 18 in <cell line: 2>()
     10     model.cache_batched_data(valid, "valid", num_samples=valid_samples)
     12 sess.run(tf.global_variables_initializer())
---> 14 model.fit()

File ~/cbcv/tft_google_ver4/libs/tft_model.py:1158, in TemporalFusionTransformer.fit(self, train_df, valid_df)
   1154 val_data, val_labels, val_flags = _unpack(valid_data)
   1156 all_callbacks = callbacks
-> 1158 self.model.fit(
   1159     x=data,
   1160     y=np.concatenate([labels, labels, labels], axis=-1),
   1161     sample_weight=active_flags,
   1162     epochs=self.num_epochs,
   1163     batch_size=self.minibatch_size,
   1164     validation_data=(val_data,
   1165                      np.concatenate([val_labels, val_labels, val_labels],
   1166                                     axis=-1), val_flags),
   1167     callbacks=all_callbacks,
   1168     shuffle=True,
   1169     use_multiprocessing=True,
   1170     workers=self.n_multiprocessing_workers)
   1172 # Load best checkpoint again
   1173 tmp_checkpont = self.get_keras_saved_path(self._temp_folder)

File /home/jhan405/miniconda3/envs/tf-gpu/lib/python3.9/site-packages/keras/engine/training_v1.py:777, in Model.fit(self, x, y, batch_size, epochs, verbose, callbacks, validation_split, validation_data, shuffle, class_weight, sample_weight, initial_epoch, steps_per_epoch, validation_steps, validation_freq, max_queue_size, workers, use_multiprocessing, **kwargs)
    774 self._check_call_args('fit')
    776 func = self._select_training_loop(x)
--> 777 return func.fit(
    778     self,
    779     x=x,
    780     y=y,
    781     batch_size=batch_size,
    782     epochs=epochs,
    783     verbose=verbose,
    784     callbacks=callbacks,
    785     validation_split=validation_split,
    786     validation_data=validation_data,
    787     shuffle=shuffle,
    788     class_weight=class_weight,
    789     sample_weight=sample_weight,
    790     initial_epoch=initial_epoch,
    791     steps_per_epoch=steps_per_epoch,
    792     validation_steps=validation_steps,
    793     validation_freq=validation_freq,
    794     max_queue_size=max_queue_size,
    795     workers=workers,
    796     use_multiprocessing=use_multiprocessing)

File /home/jhan405/miniconda3/envs/tf-gpu/lib/python3.9/site-packages/keras/engine/training_arrays_v1.py:641, in ArrayLikeTrainingLoop.fit(self, model, x, y, batch_size, epochs, verbose, callbacks, validation_split, validation_data, shuffle, class_weight, sample_weight, initial_epoch, steps_per_epoch, validation_steps, validation_freq, **kwargs)
    637     raise ValueError('`validation_steps` should not be specified if '
    638                      '`validation_data` is None.')
    639   val_x, val_y, val_sample_weights = None, None, None
--> 641 return fit_loop(
    642     model,
    643     inputs=x,
    644     targets=y,
    645     sample_weights=sample_weights,
    646     batch_size=batch_size,
    647     epochs=epochs,
    648     verbose=verbose,
    649     callbacks=callbacks,
    650     val_inputs=val_x,
    651     val_targets=val_y,
    652     val_sample_weights=val_sample_weights,
    653     shuffle=shuffle,
    654     initial_epoch=initial_epoch,
    655     steps_per_epoch=steps_per_epoch,
    656     validation_steps=validation_steps,
    657     validation_freq=validation_freq,
    658     steps_name='steps_per_epoch')

File /home/jhan405/miniconda3/envs/tf-gpu/lib/python3.9/site-packages/keras/engine/training_arrays_v1.py:377, in model_iteration(model, inputs, targets, sample_weights, batch_size, epochs, verbose, callbacks, val_inputs, val_targets, val_sample_weights, shuffle, initial_epoch, steps_per_epoch, validation_steps, validation_freq, mode, validation_in_fit, prepared_feed_values_from_dataset, steps_name, **kwargs)
    374 callbacks._call_batch_hook(mode, 'begin', batch_index, batch_logs)
    376 # Get outputs.
--> 377 batch_outs = f(ins_batch)
    378 if not isinstance(batch_outs, list):
    379   batch_outs = [batch_outs]

File /home/jhan405/miniconda3/envs/tf-gpu/lib/python3.9/site-packages/keras/backend.py:4275, in GraphExecutionFunction.__call__(self, inputs)
   4269 if (self._callable_fn is None or feed_arrays != self._feed_arrays or
   4270     symbol_vals != self._symbol_vals or
   4271     feed_symbols != self._feed_symbols or self.fetches != self._fetches or
   4272     session != self._session):
   4273   self._make_callable(feed_arrays, feed_symbols, symbol_vals, session)
-> 4275 fetched = self._callable_fn(*array_vals,
   4276                             run_metadata=self.run_metadata)
   4277 self._call_fetch_callbacks(fetched[-len(self._fetches):])
   4278 output_structure = tf.nest.pack_sequence_as(
   4279     self._outputs_structure,
   4280     fetched[:len(self.outputs)],
   4281     expand_composites=True)

File /home/jhan405/miniconda3/envs/tf-gpu/lib/python3.9/site-packages/tensorflow/python/client/session.py:1480, in BaseSession._Callable.__call__(self, *args, **kwargs)
   1478 try:
   1479   run_metadata_ptr = tf_session.TF_NewBuffer() if run_metadata else None
-> 1480   ret = tf_session.TF_SessionRunCallable(self._session._session,
   1481                                          self._handle, args,
   1482                                          run_metadata_ptr)
   1483   if run_metadata:
   1484     proto_data = tf_session.TF_GetBuffer(run_metadata_ptr)

InternalError: 2 root error(s) found.
  (0) INTERNAL: cuDNN launch failure : input shape ([1,16,160,1])
	 [[{{node TemporalFusionTransformer/layer_normalization_2/FusedBatchNormV3}}]]
	 [[TemporalFusionTransformer/dropout_10/cond/then/_150/dropout/Mul/_2045]]
  (1) INTERNAL: cuDNN launch failure : input shape ([1,16,160,1])
	 [[{{node TemporalFusionTransformer/layer_normalization_2/FusedBatchNormV3}}]]
0 successful operations.
0 derived errors ignored.
```

## Solution

Just try decrease the batch size, it will work (If all the GPU memory is used(nvidia-smi)). Else do this os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'. I had same issue which got solved with this.

https://github.com/DeepLabCut/DeepLabCut/issues/1

`os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'`



https://github.com/isl-org/Open3D-ML/issues/292

I faced this issue on Ubuntu 18.04, CUDA 11.0.3, cuDNN 8, Nvidia driver v470.103.01.

I upgraded to CUDA 11.4.3 with cuDNN 8, the issue disappeared.

=======

This may be related to the TensorFlow issue tensorflow/tensorflow#45779
and can be resolved by asking TensorFlow to allocate GPU memory only as needed:
https://www.tensorflow.org/guide/gpu#limiting_gpu_memory_growth

One way to do this is by setting the environment variable TF_FORCE_GPU_ALLOW_GROWTH to true. For example:

TF_FORCE_GPU_ALLOW_GROWTH=true python -m pytest tests/test_models.py

In [1]:
%conda list cudatoolkit

# packages in environment at /home/jhan405/miniconda3/envs/tf-gpu:
#
# Name                    Version                   Build  Channel
cudatoolkit               10.1.243             h6bb024c_0  

Note: you may need to restart the kernel to use updated packages.


In [2]:
%conda list cudnn

# packages in environment at /home/jhan405/miniconda3/envs/tf-gpu:
#
# Name                    Version                   Build  Channel
cudnn                     7.6.5                cuda10.1_0  

Note: you may need to restart the kernel to use updated packages.


In [3]:
%nvcc --vsersion

# nvcc: NVIDIA (R) Cuda compiler driver
# Copyright (c) 2005-2021 NVIDIA Corporation
# Built on Mon_Oct_11_21:27:02_PDT_2021
# Cuda compilation tools, release 11.4, V11.4.152
# Build cuda_11.4.r11.4/compiler.30521435_0

UsageError: Line magic function `%nvcc` not found.


In [5]:
import tensorflow.python.platform.build_info as build
print(build.build_info)

OrderedDict([('cpu_compiler', '/dt9/usr/bin/gcc'), ('cuda_compute_capabilities', ['sm_35', 'sm_50', 'sm_60', 'sm_70', 'sm_75', 'compute_80']), ('cuda_version', '11.2'), ('cudnn_version', '8'), ('is_cuda_build', True), ('is_rocm_build', False), ('is_tensorrt_build', True)])


# Tensorflow message

import tensorflow.compat.v1 as tf

```
2023-02-12 21:07:41.004000: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.

2023-02-12 21:07:46.421928: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered

2023-02-12 21:08:04.508297: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib:/home/jhan405/miniconda3/envs/google-tft-tf2/lib/

2023-02-12 21:08:04.509132: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib:/home/jhan405/miniconda3/envs/google-tft-tf2/lib/

2023-02-12 21:08:04.509156: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.

```

default_keras_session = tf.keras.backend.get_session()

```
WARNING:tensorflow:From /nethome/jhan405/cbcv/tft_google_ver4/jianlin-tft/script_train_fixed_params.py:99: The name tf.keras.backend.get_session is deprecated. Please use tf.compat.v1.keras.backend.get_session instead.

2023-02-13 05:12:52.784585: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.

2023-02-13 05:12:57.930672: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 17462 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:06:00.0, compute capability: 7.0
2023-02-13 05:12:57.932025: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 30975 MB memory:  -> device: 1, name: Tesla V100-SXM2-32GB, pci bus id: 0000:07:00.0, compute capability: 7.0
2023-02-13 05:12:57.933497: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 30975 MB memory:  -> device: 2, name: Tesla V100-SXM2-32GB, pci bus id: 0000:0a:00.0, compute capability: 7.0
2023-02-13 05:12:57.934443: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:3 with 30975 MB memory:  -> device: 3, name: Tesla V100-SXM2-32GB, pci bus id: 0000:0b:00.0, compute capability: 7.0
2023-02-13 05:12:57.935380: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:4 with 30975 MB memory:  -> device: 4, name: Tesla V100-SXM2-32GB, pci bus id: 0000:85:00.0, compute capability: 7.0
2023-02-13 05:12:57.936340: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:5 with 30975 MB memory:  -> device: 5, name: Tesla V100-SXM2-32GB, pci bus id: 0000:86:00.0, compute capability: 7.0
2023-02-13 05:12:57.937437: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:6 with 30975 MB memory:  -> device: 6, name: Tesla V100-SXM2-32GB, pci bus id: 0000:89:00.0, compute capability: 7.0
2023-02-13 05:12:57.938588: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:7 with 30975 MB memory:  -> device: 7, name: Tesla V100-SXM2-32GB, pci bus id: 0000:8a:00.0, compute capability: 7.0
```